In [70]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [183]:
df = pandas.read_csv('dt_all_transactions.csv')

In [196]:
df["type"] = df["type"].astype('category')
df["operation"] = df["operation"].astype('category')
df["bank"] = df["bank"].astype('category')
#df["trans_date"] = df["trans_date"].astype('datetime64')
df["account"] = df["account"].fillna(-1)
df["CRIMES_95"] = df["CRIMES_95"].astype('float').fillna(-1)

In [186]:
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

pandas.core.series.Series

In [187]:
print(df)

        Unnamed: 0  district_id  account_id  trans_id trans_date  type  \
0                1           18           1         1 1995-03-24     0   
1                2           18           1         5 1995-04-13     0   
2                3           18           1       199 1995-04-23     0   
3                4           18           1   3530438 1995-04-30     0   
4                5           18           1         6 1995-05-13     0   
5                6           18           1       200 1995-05-23     0   
6                7           18           1   3530439 1995-05-31     0   
7                8           18           1         7 1995-06-13     0   
8                9           18           1       201 1995-06-22     1   
9               10           18           1   3530440 1995-06-30     0   
10              11           18           1         8 1995-07-13     0   
11              12           18           1       202 1995-07-22     1   
12              13           18       

In [188]:
rng = np.random.RandomState(42)

In [189]:
def IsolationForestTrain(X_train, cont=0.1):
    rng = check_random_state(42)
    clf = IsolationForest(max_samples=100, random_state=rng, contamination=cont)
    clf.fit(X_train)
    y_pred_train = clf.predict(X_train)
    return y_pred_train


In [231]:
X_train = np.array(df[["account_id", "bank", "type", "operation", "amount", "balance", "account", "AVG_SALARY","UNEMP_96", "ENTR", "CRIMES_96", "last_trans"]])
Y_train_sars1 = np.array(df[["SARS_layer1"]])
Y_train_sars2 = np.array(df[["SARS_layer2"]])
Y_train_sars3 = np.array(df[["SARS_layer3"]])
Y_train_sars1[Y_train_sars1 == 0] = -1
Y_train_sars2[Y_train_sars2 == 0] = -1
Y_train_sars3[Y_train_sars3 == 0] = -1
y_pred_train = IsolationForestTrain(X_train)

In [232]:
print(list(y_pred_train).count(1))
print(list(y_pred_train).count(-1))

449999
50001


In [233]:
print(list(Y_train_sars3).count(1))
print(list(Y_train_sars3).count(-1))

48626
451374


In [234]:
compare = []
inverse = []
for i in range(len(Y_train_sars3)):
    compare.append( Y_train_sars3[i][0] == y_pred_train[i] )
    inverse.append( Y_train_sars3[i][0] != y_pred_train[i] )
print(compare.count(1))
print(inverse.count(1))

88749
411251


In [236]:
y_final_pred = y_pred_train
if compare.count(1) < inverse.count(1):
    #y_final_pred[y_pred_train == 1] = 0
    #y_final_pred[y_pred_train == -1] = 1
    y_final_pred = [ 1 if value == -1 else 0 for value in y_pred_train ]
else:
    y_final_pred = [ -1 if value == 0 else 1 for value in y_pred_train ]

In [237]:
print(list(y_final_pred).count(1))
print(list(y_final_pred).count(0))

50001
449999


In [244]:
compare_final = []
for i in range(len(y_final_pred)):
    compare_final.append( y_final_pred[i] == y_pred_train[i] )

print(list(compare_final).count(1)) #Expected 0 if there was a change in the outco

0


In [242]:
print df2

        Unnamed: 0  district_id  account_id  trans_id  trans_date        type  \
0                1           18           1         1  1995-03-24      CREDIT   
1                2           18           1         5  1995-04-13      CREDIT   
2                3           18           1       199  1995-04-23      CREDIT   
3                4           18           1   3530438  1995-04-30      CREDIT   
4                5           18           1         6  1995-05-13      CREDIT   
5                6           18           1       200  1995-05-23      CREDIT   
6                7           18           1   3530439  1995-05-31      CREDIT   
7                8           18           1         7  1995-06-13      CREDIT   
8                9           18           1       201  1995-06-22  WITHDRAWAL   
9               10           18           1   3530440  1995-06-30      CREDIT   
10              11           18           1         8  1995-07-13      CREDIT   
11              12          

In [241]:
df2 = pandas.read_csv('dt_all_transactions.csv')
df2["predicted_anomaly"] = pandas.Series(y_final_pred)
df2.to_csv("output.csv")
##END

In [147]:
print(list(y_pred_train).count(1))
print(list(y_pred_train).count(-1))
compare = []
compare_small = []
compare_small_vs_compare = []
for i in range(len(Y_train)):
    compare.append( Y_train[i][0] == y_pred_train[i] )
    compare_small.append( Y_train[i][0] == y_pred_train[i] )
    compare_small_vs_compare.append( y_pred_train_small[i] == y_pred_train[i] )

950688
105632


In [148]:
print("With the big set of parameters, we get {} matches and {} suspicious transaction").format(compare.count(1), list(y_pred_train).count(1))
print("With the small set of parameters, we get {} matches and {} suspicious transactions").format(compare_small.count(1), list(y_pred_train_small).count(1))
print("With the big set of parameters vs the small set, we get {} matches and {} differences").format(compare_small_vs_compare.count(1), compare_small_vs_compare.count(0))

With the big set of parameters, we get 190228 matches and 950688 suspicious transaction
With the small set of parameters, we get 190228 matches and 950688 suspicious transactions
With the big set of parameters vs the small set, we get 979356 matches and 76964 differences


In [157]:
# Now I compare the outcome of the 
contamination_sets = [0.01, 0.02, 0.05, 0.2]

for j in contamination_sets:
    y_pred = IsolationForestTrain(X_train, cont=j)
    aux = []
    for i in range(len(Y_train)):
        aux.append( Y_train[i][0] == y_pred[i] )
    print("With contamination = {}, we get {} matches and {} suspicious transaction")\
    .format(j, aux.count(1), list(y_pred).count(1))
    

With contamination = 0.01, we get 114142 matches and 1045756 suspicious transaction
With contamination = 0.02, we get 122521 matches and 1035193 suspicious transaction
With contamination = 0.05, we get 147981 matches and 1003503 suspicious transaction
With contamination = 0.2, we get 274736 matches and 845056 suspicious transaction


In [ ]:
df["predicted_anomaly"] = pandas.Series(y_pred_train)


In [99]:
df.to_csv("output.csv")

In [69]:
from sklearn.utils import check_random_state

rng = check_random_state(0)

def test_iforest_performance():
    """Test Isolation Forest performs well"""

    # Generate train/test data
    rng = check_random_state(2)
    X = 0.3 * rng.randn(120, 2)
    X_train = np.r_[X + 2, X - 2]
    X_train = X[:100]

    # Generate some abnormal novel observations
    X_outliers = rng.uniform(low=-4, high=4, size=(20, 2))
    X_test = np.r_[X[100:], X_outliers]
    y_test = np.array([0] * 20 + [1] * 20)

    # fit the model
    clf = IsolationForest(max_samples=100, random_state=rng).fit(X_train)

    # predict scores (the lower, the more normal)
    y_pred = - clf.decision_function(X_test)

    # check that there is at most 6 errors (false positive or false negative)
    #assert_greater(roc_auc_score(y_test, y_pred), 0.98)
    print(len(list(y_pred[y_pred < 0])))
    print(len(y_pred))
    print(y_test)
    
test_iforest_performance()

18
40
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


In [169]:
pandas.Timestamp(datetime(2012, 5, 1))

NameError: name 'datetime' is not defined